In [ ]:
# ============================================================================
# HÜCRE -1: KÜTÜPHANE GÜNCELLEME (HER ŞEYDEN ÖNCE BUNU ÇALIŞTIR)
# ============================================================================
print("Tüm gerekli kütüphaneler en güncel versiyonlara yükseltiliyor...")
print("Bu işlem birkaç dakika sürebilir...")

# '!' işareti, bu komutun bir Python kodu değil, bir terminal komutu olduğunu belirtir.
!pip install --upgrade --quiet google-generativeai langchain-google-genai langchain langchain-chroma langchain-huggingface

print("\n✅ Güncelleme tamamlandı! LÜTFEN BİR SONRAKİ ADIMA GEÇİN.")

In [ ]:
# ============================================================================
# HÜCRE 0: MODEL TEŞHİS ARACI
# Bu kod, API anahtarınızla hangi Gemini modellerini kullanabileceğinizi listeler.
# ============================================================================
import google.generativeai as genai
import os

print("API Anahtarınız için kullanılabilir Gemini modelleri listeleniyor...")
print("="*60)

try:
    api_key = os.environ.get('GOOGLE_API_KEY')
    if not api_key:
        raise ValueError("API Anahtarı bulunamadı! Lütfen 'GOOGLE_API_KEY' ortam değişkenini kontrol edin.")
    
    genai.configure(api_key=api_key)

    # API'den, 'generateContent' metodunu destekleyen tüm modelleri listelemesini isteyelim.
    model_found = False
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(f"✅ Model Adı: {m.name}")
            print(f"   Açıklama: {m.description}\n")
            model_found = True
            
    if not model_found:
        print("❌ 'generateContent' metodunu destekleyen hiçbir model bulunamadı.")
        print("   Lütfen Google AI Studio veya Google Cloud Console üzerinden projenizi ve API anahtarınızı kontrol edin.")

except Exception as e:
    print(f"🚨 Modeller listelenirken bir hata oluştu: {e}")

print("="*60)
print("Teşhis tamamlandı. Lütfen yukarıdaki 'Model Adı' listesinden birini kopyalayın.")

In [ ]:
"""
# DENEME 1: Google Gemini API ile Veri Üretme (İlk Hata)

# HEDEF: Google Gemini API'sinin ücretsiz katmanını kullanarak cevap varyasyonları üretmek.
#
# KARŞILAŞILAN SORUN: Günlük Kota Hatası (429 ResourceExhausted)
#   - Hata Mesajı: "Quota exceeded for metric: ...generate_content_free_tier_requests, limit: 200"
#   - TEŞHİS: Script bir süre (yaklaşık 90-100 soru) çalıştıktan sonra duruyordu.
#     Bunun sebebinin, Google'ın ücretsiz API katmanına koyduğu "Günlük Toplam İstek Limiti"
#     olduğu anlaşıldı. Dakikalarca beklemek bu sorunu çözmüyordu çünkü limit günlük olarak sıfırlanıyordu.
#
#   - ÇÖZÜM PLANI: Farklı APİ KEY'Ler alarak proje içerisinde 6 farklı APİ KEY Kullanıldı.


# DENEME 2: Hugging Face API'ye Geçiş (Kütüphane ve Model Hataları)

# HEDEF: Google'ın günlük limitini aşmak için Hugging Face'in ücretsiz Inference API'ını kullanmak.
#        Model olarak Meta'nın güçlü Llama-3 modeli seçildi.
#
# KARŞILAŞILAN SORUN 1: Kütüphane Uyumluluk Hatası ('thinking' Hatası)
#   - Hata Mesajı: "Model.__init__() got an unexpected keyword argument 'thinking'"
#   - TEŞHİS: Bu hatanın temel sebebi, `google-generativeai` ve `langchain` gibi
#     kütüphanelerin bilgisayarımızda yüklü olan versiyonlarının eski olmasıydı.
#     API'dan gelen yeni bir alanı tanıyamıyorlardı.
#   - ÇÖZÜM: Projenin en başına bir kurulum hücresi eklenerek tüm kütüphaneler
#     `!pip install --upgrade` komutuyla en güncel sürümlerine yükseltildi.
#
# KARŞILAŞILAN SORUN 2: Model Görev Uyumsuzluğu Hatası
#   - Hata Mesajı: "Model ... is not supported for task text-generation... Supported task: conversational"
#   - TEŞHİS: LangChain'deki `HuggingFaceEndpoint` aracının, Llama-3-Instruct gibi
#     modelleri "sohbet" formatı yerine "düz metin üretme" formatında çağırmaya çalıştığı anlaşıldı.
#
# KARŞILAŞILAN SORUN 3: Yanlış Sarmalayıcı (Wrapper) Kullanımı
#   - Hata Mesajı: "ValidationError: ... llm Field required"
#   - TEŞHİS: `ChatHuggingFace` aracının, doğrudan model bilgileriyle değil, önce
#     oluşturulmuş bir `HuggingFaceEndpoint` nesnesini `llm=` parametresi olarak beklediği
#     tespit edildi. Kod bu yapıya uygun değildi.
#
#   - ÇÖZÜM PLANI: Bu üç sorunu çözmek için kod, temel bağlantıyı `HuggingFaceEndpoint` ile
#     kuracak ve ardından bu bağlantıyı `ChatHuggingFace` sarmalayıcısına verecek şekilde
#     yeniden düzenlendi. Bu, hem doğru görev formatını (`conversational`) kullanmamızı sağladı
#     hem de kütüphanenin beklediği doğru kurulumu gerçekleştirdi.

# DENEME 3: Hugging Face ile Uzun Süreli Çalışma (Aylık Kredi Hatası)

# HEDEF: Düzeltilmiş Hugging Face koduyla tüm veri setini işlemek.
#
# KARŞILAŞILAN SORUN: Ödeme Gerekli Hatası (402 Payment Required)
#   - Hata Mesajı: "You have exceeded your monthly included credits for Inference Providers."
#   - TEŞHİS: Script'in bir süre sorunsuz çalıştıktan sonra durması, bu sefer de
#     Hugging Face'in "aylık ücretsiz kullanım kredisinin" bittiğini gösterdi.
#     Bu, günlük sıfırlanan bir limit değildi ve bu yolu da tıkamış oldu.
#     Bu durum, ücretsiz katmanların büyük ölçekli toplu işlemler için uygun
#     olmadığını, daha çok deneme ve düşük trafikli uygulamalar için tasarlandığını kanıtladı.


"""




import os  
import json  
import time  

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

print(" Sentetik Cevap Üretici (Hugging Face - Güvenli Mod) Başlatılıyor...")



INPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"
OUTPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\generated_data_hf.jsonl"
VARIANTS_PER_QUESTION = 2 

DELAY_SECONDS = 25 

HF_MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct" 


hf_token = os.environ.get('HUGGINGFACE_HUB_TOKEN')
if not hf_token:
    
    raise ValueError("Hugging Face API Token bulunamadı! Lütfen 'HUGGINGFACE_HUB_TOKEN' ortam değişkenini ayarlayın.")

endpoint_llm = HuggingFaceEndpoint(
    repo_id=HF_MODEL_NAME,  
    huggingfacehub_api_token=hf_token, 
    temperature=0.7,  
    max_new_tokens=256,  
    repetition_penalty=1.2,  
)

llm = ChatHuggingFace(llm=endpoint_llm)

print(f" Hugging Face Endpoint ve '{HF_MODEL_NAME}' modeli yapılandırıldı.")


prompt_template_str = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
Sen bir metin yeniden yazma asistanısın. Görevin, verilen bir cevabı, anlamını tamamen koruyarak ama farklı kelimelerle yeniden yazmaktır. Sadece ve sadece yeniden yazılmış cevabı döndür. Başka hiçbir açıklama ekleme.<|eot_id|><|start_header_id|>user<|end_header_id|>
Orijinal Soru: {soru}
Orijinal Cevap: {orijinal_cevap}
Yeniden Yazılmış Cevap:<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""
prompt = ChatPromptTemplate.from_template(prompt_template_str)
output_parser = StrOutputParser()

chain = prompt | llm | output_parser
print(" LangChain zinciri oluşturuldu.")


try:
    start_index = 0
    if os.path.exists(OUTPUT_FILE_PATH):
        with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as f:
            lines_in_output = len(f.readlines())
        start_index = lines_in_output // VARIANTS_PER_QUESTION
        print(f"\n Mevcut dosyada {lines_in_output} varyasyon bulundu.")
        if start_index > 0:
            print(f" İşleme {start_index + 1}. sorudan devam edilecek.")

    with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as infile, \
         open(OUTPUT_FILE_PATH, 'a', encoding='utf-8') as outfile:
        
        lines = infile.readlines()
        lines_to_process = lines[start_index:]
        
        total_lines = len(lines)
        print(f" Orijinal dosyada toplam {total_lines} soru var. Kalan {len(lines_to_process)} soru işlenecek.")

        if not lines_to_process:
            print("Tüm sorular zaten işlenmiş. İşlem tamamlandı.")
        else:
            print(f"Her API çağrısı arasında {DELAY_SECONDS} saniye beklenecektir...")

        for i, line in enumerate(lines_to_process, start=start_index):
            original_data = json.loads(line)
            question = original_data.get("question")
            answer = original_data.get("answer")
            if not question or not answer: continue

            print(f"\n[{i+1}/{total_lines}] İşleniyor: '{question[:50]}...'")
            for j in range(VARIANTS_PER_QUESTION):
                print(f"  -> Varyasyon {j+1}/{VARIANTS_PER_QUESTION} üretiliyor...")
                try:
                    new_answer = chain.invoke({
                        "soru": question,
                        "orijinal_cevap": answer
                    })
                    
                    new_data_entry = {"question": question, "answer": new_answer.strip()}
                    outfile.write(json.dumps(new_data_entry, ensure_ascii=False) + "\n")

                except Exception as e:
                    print(f"    !! API Hatası: {e}. Bu varyasyon atlanıyor.")
                    if "quota" in str(e).lower() or "payment required" in str(e).lower():
                        print("    !! KOTA LİMİTİNE ULAŞILDI. İşlem durduruluyor. Lütfen daha sonra tekrar deneyin.")
                
                time.sleep(DELAY_SECONDS)

    print("\n" + "="*80 + f"\nİŞLEM TAMAMLANDI! Üretilen veriler '{OUTPUT_FILE_PATH}' dosyasına kaydedildi.\n" + "="*80)

except Exception as e:
    print(f"\nİşlem durduruldu. Üretilen veriler kaydedildi. Limitin sıfırlanmasını bekledikten sonra aynı hücreyi tekrar çalıştırarak devam edebilirsiniz.")

In [ ]:
import json
from collections import defaultdict

INPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"
REPORT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\data_quality_report.txt"

answers_to_questions = defaultdict(list)

print(f"'{INPUT_FILE_PATH}' dosyası analiz ediliyor...")

try:
    with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                data = json.loads(line)
                if 'question' in data and 'answer' in data:
                    answers_to_questions[data['answer']].append(data['question'])
            except json.JSONDecodeError:
                print(f"Uyarı: Bir satır okunamadı, geçiliyor.")
                continue

    print("Analiz tamamlandı. Rapor oluşturuluyor...")

    with open(REPORT_FILE_PATH, 'w', encoding='utf-8') as f:
        sorted_answers = sorted(answers_to_questions.items(), key=lambda item: len(item[1]), reverse=True)
        
        for answer, questions in sorted_answers:
            f.write("="*80 + "\n")
            f.write(f"CEVAP (Toplam {len(questions)} soru için kullanılmış):\n")
            f.write(f"{answer}\n")
            f.write("-" * 20 + "\n")
            f.write("Bu Cevabın Atandığı Sorular:\n")
            for i, question in enumerate(questions, 1):
                f.write(f"  {i}. {question}\n")
            f.write("\n")

    print(f"Rapor başarıyla '{REPORT_FILE_PATH}' dosyasına kaydedildi.")
    print("Lütfen bu dosyayı açıp mantıksız Soru-Cevap eşleşmelerini kontrol et.")

except FileNotFoundError:
    print(f"HATA: Dosya bulunamadı: {INPUT_FILE_PATH}")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")

In [ ]:
import os
import json
import time

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

print("Sentetik Cevap Üretici (Google Gemini Versiyonu) Başlatılıyor...")

INPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"
OUTPUT_FILE_PATH = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\generated_data_google.jsonl"
VARIANTS_PER_QUESTION = 2 
DELAY_SECONDS = 15 
LLM_MODEL_NAME = "models/gemini-2.0-flash" 

api_key = os.environ.get('GOOGLE_API_KEY-cosmo')
if not api_key:
    raise ValueError("Google API Anahtarı bulunamadı! Lütfen 'GOOGLE_API_KEY' ortam değişkenini ayarlayın.")

llm = ChatGoogleGenerativeAI(model=LLM_MODEL_NAME, temperature=0.7, google_api_key=api_key)
print(f" LangChain ve Gemini modeli ({LLM_MODEL_NAME}) veri üretimi için yapılandırıldı.")

prompt_template_str = """
Sen bir metin yeniden yazma asistanısın. Görevin, verilen bir cevabı, anlamını tamamen koruyarak ama farklı kelimelerle yeniden yazmaktır. Sadece ve sadece yeniden yazılmış cevabı döndür. Başka hiçbir açıklama ekleme.

Orijinal Soru: {soru}
Orijinal Cevap: {orijinal_cevap}
Yeniden Yazılmış Cevap:
"""
prompt = ChatPromptTemplate.from_template(prompt_template_str)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
print("LangChain zinciri oluşturuldu.")

start_index = 0
if os.path.exists(OUTPUT_FILE_PATH):
    with open(OUTPUT_FILE_PATH, 'r', encoding='utf-8') as f:
        lines_in_output = len(f.readlines())
        start_index = lines_in_output // VARIANTS_PER_QUESTION
        print(f"\n Mevcut dosyada {lines_in_output} varyasyon bulundu.")
        if start_index > 0:
            print(f" İşleme {start_index + 1}. sorudan devam edilecek.")

try:
    with open(INPUT_FILE_PATH, 'r', encoding='utf-8') as infile, \
         open(OUTPUT_FILE_PATH, 'a', encoding='utf-8') as outfile:
        
        lines = infile.readlines()
        lines_to_process = lines[start_index:]
        total_lines = len(lines)
        
        print(f" Orijinal dosyada toplam {total_lines} soru var. Kalan {len(lines_to_process)} soru işlenecek.")
        if not lines_to_process:
            print("Tüm sorular zaten işlenmiş. İşlem tamamlandı.")
        else:
            print("İşlem başlıyor...")

        for i, line in enumerate(lines_to_process, start=start_index):
            original_data = json.loads(line)
            question = original_data.get("question")
            answer = original_data.get("answer")
            if not question or not answer: continue

            print(f"\n[{i+1}/{total_lines}] İşleniyor: '{question[:50]}...'")
            for j in range(VARIANTS_PER_QUESTION):
                print(f"  -> Varyasyon {j+1}/{VARIANTS_PER_QUESTION} üretiliyor...")
                try:
                    new_answer = chain.invoke({"soru": question, "orijinal_cevap": answer})
                    new_data_entry = {"question": question, "answer": new_answer.strip()}
                    outfile.write(json.dumps(new_data_entry, ensure_ascii=False) + "\n")
                except Exception as e:
                    print(f"    !! API Hatası: {e}. Bu varyasyon atlanıyor.")
                    if "quota" in str(e).lower():
                        print("    !! GÜNLÜK KOTA LİMİTİNE ULAŞILDI. Lütfen yarın tekrar deneyin.")
                        raise e 
                
                time.sleep(DELAY_SECONDS)

    print("\n" + "="*80 + f"\n İŞLEM TAMAMLANDI! Üretilen veriler '{OUTPUT_FILE_PATH}' dosyasına kaydedildi.\n" + "="*80)

except Exception as e:
    print(f"İşlem durduruldu: {e}")